<a href="https://colab.research.google.com/github/rosshalpin/EngineeringProject/blob/main/EngineeringProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
%cd /mydrive/EngProjV2/

/
Mounted at /content/gdrive
/content/gdrive/My Drive/EngProjV2


In [ ]:
!pip install opencv-contrib-python --upgrade

In [ ]:
%cd /mydrive/EngProjV2/

/content/gdrive/My Drive/EngProjV2


In [ ]:
import numpy as np
import cv2

In [ ]:
Conf_threshold = 0.1
NMS_threshold = 0.1
COLORS = [(0, 255, 0)]

class_name = ["cereal"]
net = cv2.dnn.readNet('/mydrive/EngProjV2/yolov4.weights', '/mydrive/EngProjV2/yolov4.cfg')
model = cv2.dnn_DetectionModel(net)
model.setInputParams(size=(416, 416), scale=1/255, swapRB=True)

In [ ]:
frame = cv2.imread("test.png")

classes, scores, boxes = model.detect(frame, Conf_threshold, NMS_threshold)
for (classid, score, box) in zip(classes, scores, boxes):
    color = COLORS[int(classid) % len(COLORS)]
    label = "%s : %f" % (class_name[0], score)
    cv2.rectangle(frame, box, color, 20)
    cv2.putText(frame, label, (box[0], box[1]-10),
                cv2.FONT_HERSHEY_PLAIN, 5, color, 5)
    cv2.imwrite("object-detection.jpg", frame)